## <a href = 'https://8weeksqlchallenge.com/case-study-2/'>Pizza Runner</a>

#### Introduction

<img src = 'https://8weeksqlchallenge.com/images/case-study-designs/2.png'>

<br>

Did you know that over 115 million kilograms of pizza is consumed daily worldwide??? (Well according to Wikipedia anyway…)

Danny was scrolling through his Instagram feed when something really caught his eye - “80s Retro Styling and Pizza Is The Future!”

Danny was sold on the idea, but he knew that pizza alone was not going to help him get seed funding to expand his new Pizza Empire - so he had one more genius idea to combine with it - he was going to Uberize it - and so Pizza Runner was launched!

Danny started by recruiting “runners” to deliver fresh pizza from Pizza Runner Headquarters (otherwise known as Danny’s house) and also maxed out his credit card to pay freelance developers to build a mobile app to accept orders from customers.

#### Case Study Questions - Part B

1. How many runners signed up for each 1 week period? (i.e. week starts 2021-01-01)
2. What was the average time in minutes it took for each runner to arrive at the Pizza Runner HQ to pickup the order?
3. Is there any relationship between the number of pizzas and how long the order takes to prepare?
4. What was the average distance travelled for each customer?
5. What was the difference between the longest and shortest delivery times for all orders?
6. What was the average speed for each runner for each delivery and do you notice any trend for these values?
7. What is the successful delivery percentage for each runner?

In [2]:
import sql
from sqlalchemy import create_engine


%load_ext sql
%sql sqlite://

Tables as per link above

In [3]:
%%sql
DROP TABLE IF EXISTS runner_orders;
CREATE TABLE runner_orders("order_id" INTEGER,
                           "runner_id" INTEGER,
                           "pickup_time" VARCHAR(19),
                           "distance" VARCHAR(7),
                           "duration" VARCHAR(10),
                           "cancellation" VARCHAR(23)
);

INSERT INTO runner_orders
    ("order_id", "runner_id", "pickup_time", "distance", "duration", "cancellation")
VALUES
    (1, 1, "2021-01-01 18:15:34", "20", "32", NULL),
    (2, 1, "2021-01-01 19:10:54", "20 km", "27 mins", NULL),
    (3, 1, "2021-01-03 00:12:37", "13.4 km", "20 mins", NULL),
    (4, 2, "2021-01-04 13:53:03", "23.4", "40", NULL),
    (5, 3, "2021-01-08 21:10:57", "10", "15", NULL),
    (6, 3, null, null, null, "Restaurant Cancellation"),
    (7, 2, "2021-01-08 21:30:45", "25 km", "25 mins", null),
    (8, 2, "2021-01-10 00:15:02", "23.4 km", "15 mins", null),
    (9, 2, null, null, null, "Customer Cancellation"),
    (10, 1, "2021-01-11 18:50:20", "10 km", "10 mins", null);

 * sqlite://
Done.
Done.
10 rows affected.


[]

In [4]:
%%sql
DROP TABLE IF EXISTS runners;
CREATE TABLE runners("runner_id" INTEGER,
                     "registration_date" DATE);
INSERT INTO runners
    ("runner_id", "registration_date")
VALUES
    (1, '2021-01-01'),
    (2, '2021-01-03'),
    (3, '2021-01-08'),
    (4, '2021-01-15');

 * sqlite://
Done.
Done.
4 rows affected.


[]

In [5]:
%%sql
DROP TABLE IF EXISTS customer_orders;
CREATE TABLE customer_orders("order_id" INTEGER,
                             "customer_id" INTEGER,
                             "pizza_id" INTEGER,
                             "exclusions" VARCHAR(4),
                             "extras" VARCHAR(4),
                             "order_time" TIMESTAMP);
INSERT INTO customer_orders
    ("order_id", "customer_id", "pizza_id", "exclusions", "extras", "order_time")
VALUES
    (1,  101, 1, " ", " ", "2021-01-01 18:05:02"),
    (2,  101, 1, " ", " ", "2021-01-01 19:00:52"),
    (3,  102, 1, " ", " ", "2021-01-02 23:51:23"),
    (3,  102, 2, " ", null, "2021-01-02 23:51:23"),
    (4,  103, 1, "4", " ", "2021-01-04 13:23:46"),
    (4,  103, 1, "4", " ", "2021-01-04 13:23:46"),
    (4,  103, 2, "4", " ", "2021-01-04 13:23:46"),
    (5,  104, 1, null, 1, "2021-01-08 21:00:29"),
    (6,  101, 2, null, null, "2021-01-08 21:03:13"),
    (7,  105, 2, null, 1, "2021-01-08 21:20:29"),
    (8,  102, 1, null, null, "2021-01-09 23:54:33"),
    (9,  103, 1, "4", "1, 5", "2021-01-10 11:22:59"),
    (10, 104, 1, null, null, "2021-01-11 18:34:49"),
    (10, 104, 1, "2, 6", "1, 4", "2021-01-11 18:34:49");

 * sqlite://
Done.
Done.
14 rows affected.


[]

In [6]:
%%sql
DROP TABLE IF EXISTS pizza_names;
CREATE TABLE pizza_names ("pizza_id" INTEGER,
                          "pizza_name" TEXT);
INSERT INTO pizza_names ("pizza_id", "pizza_name")
VALUES 
    (1, 'Meatlovers'),
    (2, 'Vegetarian');

 * sqlite://
Done.
Done.
2 rows affected.


[]

In [7]:
%%sql
DROP TABLE IF EXISTS pizza_recipes;
CREATE TABLE pizza_recipes("pizza_id" INTEGER,
                           "toppings" TEXT);
INSERT INTO pizza_recipes("pizza_id", "toppings")
VALUES
    (1, "1, 2, 3, 4, 5, 6, 8, 10"),
    (2, "4, 6, 7, 9, 11, 12");

 * sqlite://
Done.
Done.
2 rows affected.


[]

In [8]:
%%sql
DROP TABLE IF EXISTS pizza_toppings;
CREATE TABLE pizza_toppings("topping_id" INTEGER,
                            "topping_name" TEXT);
INSERT INTO pizza_toppings("topping_id", "topping_name")
VALUES
    (1, "Bacon"),
    (2, "BBQ Sauce"),
    (3, "Beef"),
    (4, "Cheese"),
    (5, "Chicken"),
    (6, "Mushrooms"),
    (7, "Onions"),
    (8, "Pepperoni"),
    (9, "Peppers"),
    (10, "Salami"),
    (11, "Tomatoes"),
    (12, "Tomato Sauce");

 * sqlite://
Done.
Done.
12 rows affected.


[]

1. How many runners signed up for each 1 week period? (i.e. week starts 2021-01-01)

In [9]:
%%sql

SELECT 
    strftime('%W', registration_date) AS Week_number,
    COUNT(runner_id) AS signups_this_week
FROM runners 
GROUP BY Week_number;

 * sqlite://
Done.


Week_number,signups_this_week
00,2
01,1
02,1


2. What was the average time in minutes it took for each runner to arrive at the Pizza Runner HQ to pickup the order?

pickup time - order time.

In [10]:
%%sql

WITH cte_time AS(
SELECT 
    ROUND(((julianday(r.pickup_time) - julianday(c.order_time))* 1440), 2) AS difference_in_mins 
FROM runner_orders AS r
JOIN customer_orders AS c
    ON r.order_id = c.order_id
WHERE r.distance <> 0)

SELECT
    ROUND(AVG(difference_in_mins), 2) AS avg_time_to_HQ
FROM cte_time
WHERE difference_in_mins >1;

 * sqlite://
Done.


avg_time_to_HQ
18.59


3. Is there any relationship between the number of pizzas and how long the order takes to prepare?

In [11]:
%%sql

WITH cte_prep AS (
SELECT 
    c.order_id,
    COUNT(c.order_id) AS number_of_pizzas,
    c.order_time,
    r.pickup_time,
    ROUND(
        (
            (julianday(r.pickup_time) - julianday(c.order_time)
            )* 1440
        ), 2
    ) AS difference_in_mins
FROM customer_orders AS c
JOIN runner_orders AS r
    ON c.order_id = r.order_id
WHERE r.distance IS NOT NULL
GROUP BY c.order_id
)

SELECT
    number_of_pizzas,
    ROUND(AVG(difference_in_mins), 1) AS avg_prep_time
FROM cte_prep
GROUP BY number_of_pizzas

 * sqlite://
Done.


number_of_pizzas,avg_prep_time
1,12.4
2,18.4
3,29.3


4. What was the average distance travelled for each customer?

In [12]:
%%sql

SELECT
    c.customer_id,
    ROUND(AVG(distance), 2) AS avg_distance_km
FROM runner_orders AS r
JOIN customer_orders AS c
    ON r.order_id = c.order_id
WHERE distance IS NOT NULL
GROUP BY c.customer_id

 * sqlite://
Done.


customer_id,avg_distance_km
101,20.0
102,16.73
103,23.4
104,10.0
105,25.0


5. What was the difference between the longest and shortest delivery times for all orders?

In [13]:
%%sql

SELECT MAX(duration) - MIN(duration) AS difference_short_long
FROM runner_orders
WHERE duration IS NOT NULL;

 * sqlite://
Done.


difference_short_long
30


6. What was the average speed for each runner for each delivery and do you notice any trend for these values?

why is speed returned as 0??

In [14]:
%%sql

SELECT
    r.runner_id, 
    r.distance,
    r.duration,
    ROUND((r.distance / r.duration), 5) AS speed,
    ROUND((r.distance/r.duration*60)) AS avg_speed_km_per_hr
FROM runner_orders AS r
WHERE r.cancellation IS NULL
ORDER BY r.runner_id

 * sqlite://
Done.


runner_id,distance,duration,speed,avg_speed_km_per_hr
1,20,32,0.0,0.0
1,20 km,27 mins,0.0,0.0
1,13.4 km,20 mins,0.67,40.0
1,10 km,10 mins,1.0,60.0
2,23.4,40,0.585,35.0
2,25 km,25 mins,1.0,60.0
2,23.4 km,15 mins,1.56,94.0
3,10,15,0.0,0.0


7. What is the successful delivery percentage for each runner?

Again, 2 + 3 should not be 0 - should be 75 % and 50 % respectively (if one includes the distance NULL entries)

In [15]:
%%sql

SELECT
    runner_id,
    100 *(SUM(CASE WHEN distance IS NULL THEN 0
             ELSE 1 
             END
       ) / count(*) 
    ) AS successfuldeliveries
FROM runner_orders
GROUP BY runner_id

 * sqlite://
Done.


runner_id,successfuldeliveries
1,100
2,0
3,0
